# Exercise - Multi step Workflow - STARTER

In this exercise, you’ll build a multi-step workflow using LCEL to solve a more complex task than simply generating a joke. 

**Challenge**

Create an AI Business Advisor that:

1. Accepts an industry as input.
2. Generates a business idea.
3. Analyzes the strengths and weaknesses.
4. Formats the results as a final report.


## 0. Import the necessary libs

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from rich.pretty import Pretty

import os

## 1. Instantiate Chat Model with your API Key

To be able to connect with OpenAI, you need to instantiate an ChatOpenAI client passing your OpenAI key.

You can pass the `api_key` argument directly.
```python
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key="voc-",
)
```

In [2]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key = os.getenv("OPEN_AI_API_KEY"),
)

## 2. Your first Chain

In the end of each chain, you should parse the output and save the logs

In [3]:
logs = []

In [4]:
parser = StrOutputParser()

In [5]:
parse_and_log_output_chain = RunnableParallel(
    output=parser, 
    log=RunnableLambda(lambda x: logs.append(x))
)

## 3. Idea Generation

Craft a prompt to generate a business idea for the given industry. 

Make sure {industry} placeholder is inside your template, so it can be filled when the chain is invoked.

In [6]:
idea_prompt = PromptTemplate(
    template=(
        "Come up with a business idea for {industry}"
    )
)
Pretty(idea_prompt)

PromptTemplate(
    input_variables=['industry'],
    input_types={},
    partial_variables={},
    template='Come up with a business idea for {industry}'
)

In [7]:
# Create your idea_chain: idea_prompt -> llm -> parse_and_log_output_chain
idea_chain = idea_prompt | llm | parse_and_log_output_chain
idea_chain

PromptTemplate(input_variables=['industry'], input_types={}, partial_variables={}, template='Come up with a business idea for {industry}')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f229c6bdeb0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f229c352120>, root_client=<openai.OpenAI object at 0x7f229ca08e30>, root_async_client=<openai.AsyncOpenAI object at 0x7f229c71f4d0>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| {
    output: StrOutputParser(),
    log: RunnableLambda(lambda x: logs.append(x))
  }

In [8]:
idea_chain.get_graph().print_ascii()

              +-------------+            
              | PromptInput |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *                  
       +---------------------------+     
       | Parallel<output,log>Input |     
       +---------------------------+     
               ***         ***           
              *               *          
            **                 ** 

In [9]:
# Test your idea_chain invoking it by passing an industy like "agro" to it
idea_result = idea_chain.invoke("argo")
Pretty(idea_result)

{
    'output': '**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven 
agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI 
(Artificial Intelligence), and data analytics to optimize farming practices. The goal is to enhance crop yield, 
reduce resource waste, and promote sustainable farming.\n\n**Key Components:**\n\n1. **IoT Sensors and Devices:**\n
- Develop a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in 
real-time.\n   - Use drones equipped with cameras and sensors to assess crop health and monitor field 
conditions.\n\n2. **Data Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from 
various sources (sensors, weather forecasts, historical data) to provide actionable insights.\n   - Use AI 
algorithms to predict crop yields, identify potential pest infestations, and recommend optimal planting and 
harvesting times.\n\n3. **Mobile App for Farmers:**\n   - Develop a user-friendly mobile app that allows farmers to
access real-time data, receive alerts, and get personalized recommendations based on their specific crops and 
conditions.\n   - Include features for tracking expenses, managing inventory, and connecting with local suppliers 
and markets.\n\n4. **Sustainability Consulting:**\n   - Offer consulting services to help farmers implement 
sustainable practices, such as crop rotation, organic farming, and water conservation techniques.\n   - Provide 
training and resources on the latest agricultural technologies and methods.\n\n5. **Subscription Model:**\n   - 
Implement a subscription-based model for farmers to access the IoT devices, data analytics platform, and consulting
services.\n   - Offer tiered pricing based on the size of the farm and the level of service required.\n\n**Target 
Market:**\n- Small to medium-sized farms looking to modernize their operations and improve efficiency.\n- 
Agricultural cooperatives and organizations focused on sustainable farming practices.\n- Government agencies and 
NGOs interested in promoting food security and sustainable agriculture.\n\n**Revenue Streams:**\n- Subscription 
fees for access to the platform and services.\n- Sales of IoT devices and sensors.\n- Consulting fees for 
sustainability and technology implementation.\n\n**Impact:**\nArgo aims to empower farmers with the tools and 
knowledge they need to increase productivity while minimizing environmental impact. By harnessing technology, Argo 
can help create a more sustainable and efficient agricultural sector, ultimately contributing to food security and 
economic growth.',
    'log': None
}

In [10]:
idea_result["output"]

'**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices. The goal is to enhance crop yield, reduce resource waste, and promote sustainable farming.\n\n**Key Components:**\n\n1. **IoT Sensors and Devices:**\n   - Develop a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time.\n   - Use drones equipped with cameras and sensors to assess crop health and monitor field conditions.\n\n2. **Data Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from various sources (sensors, weather forecasts, historical data) to provide actionable insights.\n   - Use AI algorithms to predict crop yields, identify potential pest infestations, and recommend optimal planting and harvesting times.\n\n3. **Mobile App

In [11]:
logs

[AIMessage(content='**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices. The goal is to enhance crop yield, reduce resource waste, and promote sustainable farming.\n\n**Key Components:**\n\n1. **IoT Sensors and Devices:**\n   - Develop a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time.\n   - Use drones equipped with cameras and sensors to assess crop health and monitor field conditions.\n\n2. **Data Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from various sources (sensors, weather forecasts, historical data) to provide actionable insights.\n   - Use AI algorithms to predict crop yields, identify potential pest infestations, and recommend optimal planting and harvesting times.

## 4. Idea Analysis

Craft a prompt to analyze the generated idea's strengths and weaknesses

In [12]:
analysis_prompt = PromptTemplate(
    template=(
        "Analyze the following business idea:\n" \
        "{idea}\n" \
        "List strengths and weaknesses as bullet points."
    )
)

In [13]:
analysis_chain = (
    analysis_prompt | llm | parse_and_log_output_chain
)

In [14]:
# Test your analysis_chain invoking it by passing idea_result["output"] to it
analysis_result = analysis_chain.invoke(idea_result["output"])

In [15]:
Pretty(analysis_result["output"])

'### Strengths:\n- **Innovative Technology Integration:** Utilizes cutting-edge technologies (IoT, AI, data 
analytics) to provide advanced solutions for farmers.\n- **Real-Time Monitoring:** IoT sensors enable real-time 
data collection, allowing for timely decision-making and improved crop management.\n- **Actionable Insights:** The 
data analytics platform offers predictive analytics and personalized recommendations, enhancing farming 
efficiency.\n- **User-Friendly Mobile App:** A mobile app makes it easy for farmers to access information and 
manage their operations on-the-go.\n- **Focus on Sustainability:** Emphasizes sustainable farming practices, 
appealing to environmentally conscious consumers and organizations.\n- **Diverse Revenue Streams:** Multiple 
revenue sources, including subscriptions, device sales, and consulting services, can enhance financial 
stability.\n- **Targeted Market:** Addresses the needs of small to medium-sized farms, which are often underserved 
in terms of access to technology.\n- **Potential for Scalability:** The model can be expanded to include more 
features, services, and markets as technology evolves.\n\n### Weaknesses:\n- **High Initial Costs:** Development 
and deployment of IoT devices and technology may require significant upfront investment.\n- **Technology Adoption 
Barriers:** Some farmers may be resistant to adopting new technologies due to lack of familiarity or perceived 
complexity.\n- **Dependence on Internet Connectivity:** The effectiveness of IoT devices and the data platform 
relies on stable internet access, which may be limited in rural areas.\n- **Data Privacy Concerns:** Farmers may 
have concerns about data security and privacy, which could hinder adoption.\n- **Market Competition:** The 
agricultural tech space is becoming increasingly crowded, with many competitors offering similar solutions.\n- 
**Variable ROI:** The return on investment for farmers may vary based on factors like crop type, local conditions, 
and market fluctuations.\n- **Need for Continuous Support:** Ongoing technical support and training will be 
necessary to ensure farmers can effectively use the technology, which could increase operational costs.\n- 
**Regulatory Challenges:** Navigating agricultural regulations and compliance can be complex and may pose 
challenges for implementation.'

In [16]:
logs

[AIMessage(content='**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices. The goal is to enhance crop yield, reduce resource waste, and promote sustainable farming.\n\n**Key Components:**\n\n1. **IoT Sensors and Devices:**\n   - Develop a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time.\n   - Use drones equipped with cameras and sensors to assess crop health and monitor field conditions.\n\n2. **Data Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from various sources (sensors, weather forecasts, historical data) to provide actionable insights.\n   - Use AI algorithms to predict crop yields, identify potential pest infestations, and recommend optimal planting and harvesting times.

## 5. Report Generation

Craft a prompt to structure the information into a business report.

In [17]:
# Your prompt Template
report_prompt = PromptTemplate(
    template=(
        "You are an analyst having to summarize this analysis of a business idea into a report." \
        "Analysis: {output}" 
    )
)

In [18]:
class AnalysisReport(BaseModel):
    """Strengths and Weaknesses about a business idea"""
    strengths: list = Field(default=[], description="Idea's strength list")
    weaknesses: list = Field(default=[], description="Idea's weaknesse list")

In [19]:
report_chain = (
    report_prompt | llm.with_structured_output(AnalysisReport)
)

In [20]:
# Test your report_chain invoking it by passing analysis_result["output"] to it
report_result = report_chain.invoke(analysis_result["output"])

In [21]:
Pretty(report_result)

AnalysisReport(
    strengths=[
        'Innovative Technology Integration',
        'Real-Time Monitoring',
        'Actionable Insights',
        'User-Friendly Mobile App',
        'Focus on Sustainability',
        'Diverse Revenue Streams',
        'Targeted Market',
        'Potential for Scalability'
    ],
    weaknesses=[
        'High Initial Costs',
        'Technology Adoption Barriers',
        'Dependence on Internet Connectivity',
        'Data Privacy Concerns',
        'Market Competition',
        'Variable ROI',
        'Need for Continuous Support',
        'Regulatory Challenges'
    ]
)

## 6. End to End Chain

In [22]:
e2e_chain = ( 
    RunnablePassthrough() 
    | idea_chain
    | RunnableParallel(idea=RunnablePassthrough())
    | analysis_chain
    | report_chain
)

In [23]:
e2e_chain.get_graph().print_ascii()

            +------------------+         
            | PassthroughInput |         
            +------------------+         
                      *                  
                      *                  
                      *                  
              +-------------+            
              | Passthrough |            
              +-------------+            
                      *                  
                      *                  
                      *                  
             +----------------+          
             | PromptTemplate |          
             +----------------+          
                      *                  
                      *                  
                      *                  
               +------------+            
               | ChatOpenAI |            
               +------------+            
                      *                  
                      *                  
                      *           

In [24]:
# Change the industry if you want
e2e_result = e2e_chain.invoke("agro")

In [25]:
Pretty(e2e_result)

AnalysisReport(
    strengths=[
        'Innovative Approach: Utilizes advanced technology (IoT, AI) to optimize farming processes, enhancing 
efficiency and crop yield.',
        'Sustainability Focus: Emphasizes eco-friendly practices, such as renewable energy use and waste recycling,
appealing to environmentally conscious consumers.',
        'Modular Design: Flexibility in installation allows for adaptation to various urban settings, maximizing 
space utilization.',
        'Market Demand: Targets the growing demand for fresh, locally-sourced produce in urban areas, aligning with
health trends.',
        'Community Engagement: Fosters a sense of community through educational programs and subscription services,
enhancing customer loyalty.',
        'Diverse Revenue Streams: Multiple income sources, including produce sales, subscriptions, workshops, and 
technology licensing, reduce financial risk.'
    ],
    weaknesses=[
        'High Initial Investment: The cost of technology, infrastructure, and setup may be significant, posing a 
barrier to entry.',
        'Technical Complexity: Reliance on advanced technology may lead to operational challenges and require 
specialized knowledge for maintenance and troubleshooting.',
        'Market Competition: The vertical farming industry is becoming increasingly competitive, with established 
players and new entrants.',
        'Regulatory Challenges: Urban farming may face zoning laws, health regulations, and other legal hurdles 
that could complicate operations.',
        'Consumer Awareness: The concept of vertical farming may still be unfamiliar to some consumers, requiring 
marketing efforts to educate the market.',
        'Dependency on Technology: Over-reliance on technology could lead to vulnerabilities, such as system 
failures or cyber threats.'
    ]
)

In [26]:
Pretty(e2e_result.strengths)

[
    'Innovative Approach: Utilizes advanced technology (IoT, AI) to optimize farming processes, enhancing 
efficiency and crop yield.',
    'Sustainability Focus: Emphasizes eco-friendly practices, such as renewable energy use and waste recycling, 
appealing to environmentally conscious consumers.',
    'Modular Design: Flexibility in installation allows for adaptation to various urban settings, maximizing space 
utilization.',
    'Market Demand: Targets the growing demand for fresh, locally-sourced produce in urban areas, aligning with 
health trends.',
    'Community Engagement: Fosters a sense of community through educational programs and subscription services, 
enhancing customer loyalty.',
    'Diverse Revenue Streams: Multiple income sources, including produce sales, subscriptions, workshops, and 
technology licensing, reduce financial risk.'
]

In [27]:
Pretty(e2e_result.weaknesses)

[
    'High Initial Investment: The cost of technology, infrastructure, and setup may be significant, posing a 
barrier to entry.',
    'Technical Complexity: Reliance on advanced technology may lead to operational challenges and require 
specialized knowledge for maintenance and troubleshooting.',
    'Market Competition: The vertical farming industry is becoming increasingly competitive, with established 
players and new entrants.',
    'Regulatory Challenges: Urban farming may face zoning laws, health regulations, and other legal hurdles that 
could complicate operations.',
    'Consumer Awareness: The concept of vertical farming may still be unfamiliar to some consumers, requiring 
marketing efforts to educate the market.',
    'Dependency on Technology: Over-reliance on technology could lead to vulnerabilities, such as system failures 
or cyber threats.'
]

In [28]:
logs

[AIMessage(content='**Business Idea: Argo - Smart Agricultural Solutions**\n\n**Overview:**\nArgo is a tech-driven agricultural solutions company that leverages advanced technologies such as IoT (Internet of Things), AI (Artificial Intelligence), and data analytics to optimize farming practices. The goal is to enhance crop yield, reduce resource waste, and promote sustainable farming.\n\n**Key Components:**\n\n1. **IoT Sensors and Devices:**\n   - Develop a range of IoT sensors that monitor soil moisture, temperature, humidity, and nutrient levels in real-time.\n   - Use drones equipped with cameras and sensors to assess crop health and monitor field conditions.\n\n2. **Data Analytics Platform:**\n   - Create a cloud-based platform that aggregates data from various sources (sensors, weather forecasts, historical data) to provide actionable insights.\n   - Use AI algorithms to predict crop yields, identify potential pest infestations, and recommend optimal planting and harvesting times.

## 7. Experiment

Now that you understood how it works, experiment with new things.
- Improve memory
- Explore the Runnables
- Add More Complexity